<font size=5, font color='blue'> stata_df is the dataframe downloaded through Stata API

In [8]:
import pandas as pd
pd.options.mode.chained_assignment = None

stata_df = pd.read_stata("AMECO.dta")
stata_df.head()

,period,period_start_day,value,frequency,dataset_code,dataset_name,freq,geo,unit,indexed_at,provider_code,series_code,series_name,series_num
0,1960,1960-01-01,NA,annual,ZCPIN,National consumer price index (All-items),a,alb,2015-100,2024-11-16T01:11:23.801Z,AMECO,ALB.3.0.0.0.ZCPIN,Annually – (2015 = 100) – Albania,Master
1,1961,1961-01-01,NA,annual,ZCPIN,National consumer price index (All-items),a,alb,2015-100,2024-11-16T01:11:23.801Z,AMECO,ALB.3.0.0.0.ZCPIN,Annually – (2015 = 100) – Albania,Master
2,1962,1962-01-01,NA,annual,ZCPIN,National consumer price index (All-items),a,alb,2015-100,2024-11-16T01:11:23.801Z,AMECO,ALB.3.0.0.0.ZCPIN,Annually – (2015 = 100) – Albania,Master
3,1963,1963-01-01,NA,annual,ZCPIN,National consumer price index (All-items),a,alb,2015-100,2024-11-16T01:11:23.801Z,AMECO,ALB.3.0.0.0.ZCPIN,Annually – (2015 = 100) – Albania,Master
4,1964,1964-01-01,NA,annual,ZCPIN,National consumer price index (All-items),a,alb,2015-100,2024-11-16T01:11:23.801Z,AMECO,ALB.3.0.0.0.ZCPIN,Annually – (2015 = 100) – Albania,Master


<font size=5, font color='blue'> python_df is the dataframe downloaded through Python API. 

In [9]:
python_df = pd.read_pickle("AMECO.pkl")
python_df.head()

,@frequency,provider_code,dataset_code,dataset_name,series_code,series_name,original_period,period,original_value,value,freq,unit,geo,Frequency,Unit,Country
0,annual,AMECO,XUNRQ-1,"Real effective exchange rates, based on unit l...",AUS.3.0.30.424.XUNRQ,Annually – (2015 = 100) – Australia,1960,1960-01-01,NA,NaN,a,2015-100,aus,Annually,(2015 = 100),Australia
1,annual,AMECO,XUNRQ-1,"Real effective exchange rates, based on unit l...",AUS.3.0.30.424.XUNRQ,Annually – (2015 = 100) – Australia,1961,1961-01-01,NA,NaN,a,2015-100,aus,Annually,(2015 = 100),Australia
2,annual,AMECO,XUNRQ-1,"Real effective exchange rates, based on unit l...",AUS.3.0.30.424.XUNRQ,Annually – (2015 = 100) – Australia,1962,1962-01-01,NA,NaN,a,2015-100,aus,Annually,(2015 = 100),Australia
3,annual,AMECO,XUNRQ-1,"Real effective exchange rates, based on unit l...",AUS.3.0.30.424.XUNRQ,Annually – (2015 = 100) – Australia,1963,1963-01-01,NA,NaN,a,2015-100,aus,Annually,(2015 = 100),Australia
4,annual,AMECO,XUNRQ-1,"Real effective exchange rates, based on unit l...",AUS.3.0.30.424.XUNRQ,Annually – (2015 = 100) – Australia,1964,1964-01-01,NA,NaN,a,2015-100,aus,Annually,(2015 = 100),Australia


<font size=5, font color='blue'> Check whether the two dataframes have the same number of rows, which means equal number of data points / observations. Column numbers of the two dataframes may differ due to the difference between Stata and Python APIs, and so do the order, datatypes and names of columns and the order of rows, which is why we don't require a complete match between python_df and stata_df.

In [10]:
stata_df.shape[0]==python_df.shape[0]

True

<font size=5, font color='blue'> ['period_start_day'] in stata_df is the most accurate timestamp assigned to each observation (row). In python_df its equivalent is ['period'], so we rename it to match the column name in stata_df. We match the datatype of the two columns by converting both of them to pandas datetime objects. We also convert the datatype of ['value'] in stata_df to numeric.

In [11]:
python_df.rename(columns={'period':'period_start_day'},inplace=True)
stata_df['period_start_day'] = pd.to_datetime(stata_df['period_start_day'])
python_df['period_start_day'] = pd.to_datetime(python_df['period_start_day'])
stata_df['value'] = pd.to_numeric(stata_df['value'],'coerce')

<font size=5, font color='blue'> Check whether ['period_start_day', 'series_code'] can identify a unique row in both stata_df and python_df, if not then there is a duplication or error in Dbnomics.

In [12]:
cols = ['period_start_day', 'series_code']
sum(stata_df.duplicated(subset=cols))+sum(python_df.duplicated(subset=cols)) == 0

True

<font size=5, font color='blue'> Then we sort the two dataframes by ['period_start_day', 'series_code']. This ensures observations in the two dataframes are in the same order. Finally we check whether the two sub-frames [['period_start_day','series_code','value']] sliced from stata_df and python_df are exactly the same.

In [13]:
stata_sub = stata_df.sort_values(by=cols)[cols+['value']].reset_index(drop=True)
python_sub = python_df.sort_values(by=cols)[cols+['value']].reset_index(drop=True)
stata_sub.equals(python_sub)

True